In [1]:
from __future__ import print_function
import warnings
warnings.filterwarnings('ignore')
## Disable tf future deprecated messages
import logging
logging.getLogger('tensorflow').disabled = True
## Disable tf CUDA messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

%matplotlib inline

import sys
import random
import numpy as np
import pandas as pd

import utils
import correction
from models.dense import *
from models.gain import gain
from models.soft_impute import SoftImpute
from models.sinkhorn import OTimputer
from models.mida import mida
from models.polishing import polishing
from models.filtering import filtering

import scipy.stats

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer, SimpleImputer

import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True
dtype = torch.cuda.FloatTensor

## Repeat all experiments for 10 runs

In [2]:
n_runs = 10

## Load data

In [3]:
data_missing, missing_mask, y = utils.load_nhanes()
## Inject noise
data_missing = utils.inject_noise(data_missing.copy(), noise_rate=0.)
## Replace missing values locations by 0
data_missing = data_missing * missing_mask

Dataset shape: (2000, 95)
33.65% missing data
Class distribution: (array([1., 2.]), array([1000, 1000]))


## Run OUR METHOD

In [4]:
params = {
    'nb_batches': 10,
    'reg_noise_std': .03,
    'net_input': 'data_corrupted',
    'net_params': {
        'input_channels':1,
        'output_channels':1,
        'channels_skip':4,
        'down_layers':[(24, 7, 1),
                       (46, 9, 5),
                       (96, 11, 1),
                       (96, 13, 1)],
        'need_sigmoid':True,
        'need_bias':True,
        'pad':'zero',
        'downsample_mode':'stride',
        'upsample_mode':'nearest',
        'act_fun':'LeakyReLU',
        'need1x1_up':True
    },# or a list containing layers sizes
    'adam_lr': .0001,
    'adam_weight_decay': 0.,
}

ours_accs, ours_aucs = [], []
for i in range(n_runs):
    ours_correction = correction.run(data_missing, 501, params, y=y, missing_mask=missing_mask, seed=i)
    ## Since our method returns 2 imputations select the best one
    scores_raw = utils.get_scores(ours_correction['raw_out'], y)
    acc = scores_raw['test_balanced_accuracy']
    auc = scores_raw['test_roc_auc_ovo']
    acc = acc.mean()
    acc_std = acc.std()
    auc = auc.mean()
    auc_std = auc.std()
    if 'masked_out' in ours_correction:
        scores_masked = utils.get_scores(ours_correction['masked_out'], y)
        acc_masked = scores_masked['test_balanced_accuracy']
        auc_masked = scores_masked['test_roc_auc_ovo']
        if auc_masked.mean() > auc:
            acc = acc_masked.mean()
            acc_std = acc_masked.std()
            auc = auc_masked.mean()
            auc_std = auc_masked.std()
    print(f'OUR METHOD RUN {i + 1}/{n_runs} - acc: {round(acc.mean() * 100, 2)}% +- {round(acc.std() * 100, 2)}% - ' +
          f'auc: {round(auc.mean() * 100, 2)}% +- {round(auc.std() * 100, 2)}%')
    ours_accs.append(acc.mean())
    ours_aucs.append(auc.mean())
print(f'\nOUR METHOD GLOBAL - acc: {round(np.array(ours_accs).mean() * 100, 2)}% +- {round(np.array(ours_accs).std() * 100, 2)}% - ' +
      f'auc: {round(np.array(ours_aucs).mean() * 100, 2)}% +- {round(np.array(ours_aucs).std() * 100, 2)}%')

Ite 00000 - 4.34 sec - Loss 0.060620 - ACC 62.45% - ACC Mean 62.45% - AUC 68.32% - AUC Mean 68.32% - Deter 000
Ite 00050 - 1.58 sec - Loss 0.011666 - ACC 62.05% - ACC Mean 62.21% - AUC 68.03% - AUC Mean 67.34% - Deter 025
Ite 00075 - 1.10 sec - Loss 0.008583 - ACC 60.70% - ACC Mean 62.35% - AUC 66.03% - AUC Mean 67.42% - Deter 050
Early stop ite 75, rollback to correction of ite 25, whith acc of 63.65% and auc of 69.16%
OUR METHOD RUN 1/10 - acc: 63.65% +- 0.0% - auc: 69.16% +- 0.0%
Ite 00000 - 0.92 sec - Loss 0.145551 - ACC 63.60% - ACC Mean 63.60% - AUC 68.19% - AUC Mean 68.19% - Deter 000
Ite 00050 - 1.76 sec - Loss 0.054918 - ACC 62.50% - ACC Mean 62.46% - AUC 67.57% - AUC Mean 67.58% - Deter 030
Ite 00070 - 1.00 sec - Loss 0.045855 - ACC 62.45% - ACC Mean 62.64% - AUC 67.05% - AUC Mean 67.79% - Deter 050
Early stop ite 70, rollback to correction of ite 20, whith acc of 64.75% and auc of 69.83%
OUR METHOD RUN 2/10 - acc: 64.75% +- 0.0% - auc: 69.83% +- 0.0%
Ite 00000 - 1.40 sec - L

## Run MICE

In [5]:
data_missing_nans = np.where(missing_mask, data_missing, np.nan)

imputer = IterativeImputer()
imputed = imputer.fit_transform(data_missing_nans)
## All runs would be the same since deterministic method
scores = utils.get_scores(imputed, y)
acc = [scores['test_balanced_accuracy'].mean()] * n_runs
auc = [scores['test_roc_auc_ovo'].mean()] * n_runs
print(f'MICE - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
      f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
mice_accs = acc
mice_aucs = auc

MICE - acc: 61.7% +- 0.0% - auc: 67.3% +- 0.0%


## Run SFIL

In [6]:
sfil_accs, sfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='standard', random_state=i)
    sfil_accs.append(acc.mean())
    sfil_aucs.append(auc.mean())
    print(f'SFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SFIL GLOBAL - acc: {round(np.mean(sfil_accs) * 100, 2)}% +- {round(np.std(sfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(sfil_aucs) * 100, 2)}% +- {round(np.std(sfil_aucs) * 100, 2)}%')

SFIL RUN 1/10 - acc: 61.95% +- 2.53% - auc: 66.35% +- 3.35%
SFIL RUN 2/10 - acc: 63.45% +- 1.32% - auc: 67.48% +- 1.97%
SFIL RUN 3/10 - acc: 63.45% +- 2.67% - auc: 67.94% +- 2.02%
SFIL RUN 4/10 - acc: 60.3% +- 2.96% - auc: 66.01% +- 3.38%
SFIL RUN 5/10 - acc: 63.65% +- 1.5% - auc: 68.13% +- 0.88%
SFIL RUN 6/10 - acc: 63.15% +- 1.03% - auc: 67.55% +- 0.98%
SFIL RUN 7/10 - acc: 62.6% +- 2.43% - auc: 66.58% +- 1.43%
SFIL RUN 8/10 - acc: 62.05% +- 1.39% - auc: 66.73% +- 2.31%
SFIL RUN 9/10 - acc: 61.7% +- 1.73% - auc: 67.78% +- 1.7%
SFIL RUN 10/10 - acc: 61.95% +- 2.33% - auc: 66.88% +- 2.6%
SFIL GLOBAL - acc: 62.43% +- 0.99% - auc: 67.14% +- 0.69%


## Run PFIL

In [7]:
pfil_accs, pfil_aucs = [], []
for i in range(n_runs):
    acc, auc = filtering(imputed, y, mode='panda', random_state=i)
    pfil_accs.append(acc.mean())
    pfil_aucs.append(auc.mean())
    print(f'PFIL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PFIL GLOBAL - acc: {round(np.mean(pfil_accs) * 100, 2)}% +- {round(np.std(pfil_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(pfil_aucs) * 100, 2)}% +- {round(np.std(pfil_aucs) * 100, 2)}%')

PFIL RUN 1/10 - acc: 60.05% +- 1.71% - auc: 66.0% +- 2.8%
PFIL RUN 2/10 - acc: 62.8% +- 2.3% - auc: 67.56% +- 2.06%
PFIL RUN 3/10 - acc: 62.0% +- 1.52% - auc: 65.88% +- 1.65%
PFIL RUN 4/10 - acc: 60.6% +- 1.93% - auc: 66.51% +- 2.23%
PFIL RUN 5/10 - acc: 59.75% +- 1.65% - auc: 64.84% +- 1.7%
PFIL RUN 6/10 - acc: 61.4% +- 1.89% - auc: 67.16% +- 1.59%
PFIL RUN 7/10 - acc: 60.55% +- 2.98% - auc: 66.56% +- 3.07%
PFIL RUN 8/10 - acc: 64.1% +- 1.95% - auc: 68.83% +- 2.49%
PFIL RUN 9/10 - acc: 60.55% +- 1.28% - auc: 64.54% +- 2.33%
PFIL RUN 10/10 - acc: 63.35% +- 2.82% - auc: 68.11% +- 2.71%
PFIL GLOBAL - acc: 61.52% +- 1.41% - auc: 66.6% +- 1.29%


## Run SPOL

In [8]:
spol_accs, spol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='standard', random_state=i)
    spol_accs.append(acc.mean())
    spol_aucs.append(auc.mean())
    print(f'SPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'SPOL GLOBAL - acc: {round(np.mean(spol_accs) * 100, 2)}% +- {round(np.std(spol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(spol_aucs) * 100, 2)}% +- {round(np.std(spol_aucs) * 100, 2)}%')

SPOL RUN 1/10 - acc: 61.75% +- 2.32% - auc: 65.86% +- 4.46%
SPOL RUN 2/10 - acc: 61.25% +- 1.51% - auc: 66.05% +- 1.87%
SPOL RUN 3/10 - acc: 61.0% +- 3.24% - auc: 66.14% +- 3.06%
SPOL RUN 4/10 - acc: 61.55% +- 3.04% - auc: 67.16% +- 3.72%
SPOL RUN 5/10 - acc: 63.65% +- 1.86% - auc: 68.33% +- 1.64%
SPOL RUN 6/10 - acc: 60.4% +- 1.62% - auc: 65.31% +- 1.44%
SPOL RUN 7/10 - acc: 62.5% +- 3.08% - auc: 68.32% +- 2.67%
SPOL RUN 8/10 - acc: 61.3% +- 1.95% - auc: 65.71% +- 2.47%
SPOL RUN 9/10 - acc: 61.45% +- 2.58% - auc: 66.89% +- 2.57%
SPOL RUN 10/10 - acc: 60.35% +- 2.7% - auc: 65.0% +- 2.3%
SPOL GLOBAL - acc: 61.52% +- 0.93% - auc: 66.48% +- 1.11%


## Run PPOL

In [9]:
ppol_accs, ppol_aucs = [], []
for i in range(n_runs):
    acc, auc = polishing(imputed, y, mode='panda', random_state=i)
    ppol_accs.append(acc.mean())
    ppol_aucs.append(auc.mean())
    print(f'PPOL RUN {i + 1}/{n_runs} - acc: {round(np.mean(acc) * 100, 2)}% +- {round(np.std(acc) * 100, 2)}% - ' +
          f'auc: {round(np.mean(auc) * 100, 2)}% +- {round(np.std(auc) * 100, 2)}%')
print(f'PPOL GLOBAL - acc: {round(np.mean(ppol_accs) * 100, 2)}% +- {round(np.std(ppol_accs) * 100, 2)}% - ' +
      f'auc: {round(np.mean(ppol_aucs) * 100, 2)}% +- {round(np.std(ppol_aucs) * 100, 2)}%')

PPOL RUN 1/10 - acc: 61.55% +- 2.37% - auc: 66.65% +- 2.47%
PPOL RUN 2/10 - acc: 61.2% +- 2.5% - auc: 66.17% +- 2.88%
PPOL RUN 3/10 - acc: 61.75% +- 1.34% - auc: 67.33% +- 1.51%
PPOL RUN 4/10 - acc: 61.55% +- 1.0% - auc: 67.47% +- 1.32%
PPOL RUN 5/10 - acc: 60.5% +- 1.64% - auc: 64.13% +- 2.39%
PPOL RUN 6/10 - acc: 60.45% +- 2.51% - auc: 65.5% +- 2.76%
PPOL RUN 7/10 - acc: 62.35% +- 1.1% - auc: 66.34% +- 0.96%
PPOL RUN 8/10 - acc: 62.65% +- 1.47% - auc: 67.7% +- 1.94%
PPOL RUN 9/10 - acc: 60.45% +- 2.54% - auc: 65.52% +- 2.61%
PPOL RUN 10/10 - acc: 63.75% +- 2.74% - auc: 68.1% +- 2.57%
PPOL GLOBAL - acc: 61.62% +- 1.02% - auc: 66.49% +- 1.16%


## Run T-tests

In [10]:
for model, metrics in {
        'MICE': {'ACC': mice_accs, 'AUC': mice_aucs},
        'SFIL': {'ACC': sfil_accs, 'AUC': sfil_aucs},
        'PFIL': {'ACC': pfil_accs, 'AUC': pfil_aucs},
        'SPOL': {'ACC': spol_accs, 'AUC': spol_aucs},
        'PPOL': {'ACC': ppol_accs, 'AUC': ppol_aucs}}.items():
    for metric_name, metric in metrics.items():
        ours_metric = ours_accs if metric_name == 'ACC' else ours_aucs
        t, p = scipy.stats.ttest_ind(np.array(ours_metric), np.array(metric))
        if p <= .05:
            if t > 0:
                ## Our method is better
                print(f'Metric {metric_name} - OUR METHOD is significantly better than {model}')
            else:
                ## Theirs is better
                print(f'Metric {metric_name} - OUR METHOD is significantly worse than {model}')
        else:
            ## Else we are even
            print(f'Metric {metric_name} - OUR METHOD is even with {model}')

Metric ACC - OUR METHOD is significantly better than MICE
Metric AUC - OUR METHOD is significantly better than MICE
Metric ACC - OUR METHOD is significantly better than SFIL
Metric AUC - OUR METHOD is significantly better than SFIL
Metric ACC - OUR METHOD is significantly better than PFIL
Metric AUC - OUR METHOD is significantly better than PFIL
Metric ACC - OUR METHOD is significantly better than SPOL
Metric AUC - OUR METHOD is significantly better than SPOL
Metric ACC - OUR METHOD is significantly better than PPOL
Metric AUC - OUR METHOD is significantly better than PPOL
